In [1]:
def rmmk(dir): # remove exiting folder and remake folder
    import os
    import shutil

    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.makedirs(dir)

In [7]:
def save_txt(src,dst):
    import os
    with open(os.path.join(dst), 'a', encoding ='utf8') as txt_file:
            txt_file.write(src+'\n')

In [ ]:
# import keyring
# keyring.set_password("sharepointonline_username","username","xxxx@irpc.co.th")
# keyring.set_password("sharepointonline_pass","password","xxx")
# print(keyring.get_password("sharepointonline_username","username"),keyring.get_password("sharepointonline_pass","password"))

In [ ]:
'''
/sites/FlareSmokeDetection
/sites/FlareVDO
/sites/FlareVDO1
'''

In [5]:
# https://gist.github.com/hyunjong-lee/e0f97aa0e4c353c65c31b596765ee641
def list_sharepoint_index(basesite, create_folder=False):
    import os
    from office365.runtime.auth.authentication_context import AuthenticationContext
    from office365.sharepoint.client_context import ClientContext
    from office365.sharepoint.files.file_system_object_type import FileSystemObjectType
    from office365.sharepoint.files.file import File
    from tqdm.notebook import tqdm
    import keyring

    sharepoint_user = keyring.get_password("sharepointonline_username", "username")
    sharepoint_pass = keyring.get_password("sharepointonline_pass","password")

    baseurl = 'https://irpcplc.sharepoint.com'
    basesite = basesite # every share point has a home.
    docurl = '/Shared%20Documents/Clips'
    siteurl = baseurl + basesite 
    # target_url = basesite+docurl

    ctx_auth = AuthenticationContext(siteurl)
    ctx_auth.acquire_token_for_user(sharepoint_user, sharepoint_pass)
    ctx = ClientContext(siteurl, ctx_auth) # make sure you auth to the siteurl.

    doc_lib = ctx.web.lists.get_by_title("Documents")
    items = doc_lib.items.select(["FileSystemObjectType"]).expand(["File", "Folder"]).get().execute_query()
    folders = []
    files = []
    dst_dir = './sharepointindex'
    if create_folder == True:
        rmmk(dst_dir)
    for item in tqdm(items):  # type: ListItem
        if item.file_system_object_type == FileSystemObjectType.Folder:
            print("Folder url: {0}".format(item.folder.serverRelativeUrl))
            src_folder = item.folder.serverRelativeUrl
            save_txt(src_folder,os.path.join(dst_dir,'sharepointfolders.txt'))
            folders.append(item.folder.serverRelativeUrl)
        else:
            print("File url: {0}".format(item.file.serverRelativeUrl))
            src_file = item.file.serverRelativeUrl
            save_txt(src_file,os.path.join(dst_dir,'sharepointfiles.txt'))
            files.append(item.file.serverRelativeUrl)
    return ctx, files, folders

In [18]:
ctx, files, folders = list_sharepoint_index(basesite='/sites/FlareVDO1', create_folder=True)

  0%|          | 0/100 [00:00<?, ?it/s]

Folder url: /sites/FlareVDO1/Shared Documents/Clips
Folder url: /sites/FlareVDO1/Shared Documents/Clips/tf1
Folder url: /sites/FlareVDO1/Shared Documents/Clips/seaview
Folder url: /sites/FlareVDO1/Shared Documents/Clips/tf1inf
Folder url: /sites/FlareVDO1/Shared Documents/Clips/uhv
Folder url: /sites/FlareVDO1/Shared Documents/Clips/tf2
Folder url: /sites/FlareVDO1/Shared Documents/Clips/ps
File url: /sites/FlareVDO1/Shared Documents/Clips/seaview/seaview_2023-04-05-08-41-06.mp4
File url: /sites/FlareVDO1/Shared Documents/Clips/seaview/seaview_2023-04-05-08-36-06.mp4
File url: /sites/FlareVDO1/Shared Documents/Clips/seaview/seaview_2023-04-05-08-31-06.mp4
File url: /sites/FlareVDO1/Shared Documents/Clips/seaview/seaview_2023-04-05-08-26-06.mp4
File url: /sites/FlareVDO1/Shared Documents/Clips/seaview/seaview_2023-04-05-08-21-06.mp4
File url: /sites/FlareVDO1/Shared Documents/Clips/seaview/seaview_2023-04-05-08-16-06.mp4
File url: /sites/FlareVDO1/Shared Documents/Clips/seaview/seaview_

In [19]:
filtered = filter(lambda files: '/Clips/' in files, files)
filtered = list(filtered)
folders = list(set([(f.split('/')[-2]) for f in filtered]))
for f in folders:
    rmmk(f'./Clips/{f}')

In [20]:
# https://stackoverflow.com/questions/53671547/python-download-files-from-sharepoint-site
from tqdm.notebook import tqdm
for f in tqdm(filtered):
    # print(i, f, f.split('/')[-2])
    with open(f"./Clips/{f.split('/')[-2]}/{f.split('/')[-1]}", "wb") as local_file:
        file = ctx.web.get_file_by_server_relative_url(f)
        file.download(local_file)
        ctx.execute_query()

  0%|          | 0/344 [00:00<?, ?it/s]